In [167]:
import os
import sys
import psycopg2
from sqlalchemy import create_engine
from io import StringIO
import pandas as pd
import csv
up_one =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(up_one)
up_two =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
sys.path.append(up_two)
#from minorlab import metapdb
import subprocess
from collections import defaultdict

import pprint
pp = pprint.PrettyPrinter(indent=4)




## Export cell output to a file 

Cell 1  
<code>
    %%capture cap --no-stderr
    print(q)
</code>

Cell 2  
<code>
with open('output.txt', 'w') as f:    
    f.write(cap.stdout)
    </code>

### Appris files and function

In [207]:
appris = pd.read_csv('Data/appris_transcript_names.tsv', sep='\t', header=None, names=['gene','ref'])
nonappris = pd.read_csv('Data/not_appris_transcript_names.tsv', sep='\t', header=None, names=['gene','notref'])
def ref_nonref(gene):
    ref = appris.loc [ appris['gene']== gene]['ref'].tolist()
    #print (ref)
    non_ref = nonappris.loc [nonappris['gene']==gene]['notref'].tolist()
    non_ref.sort()
    #print (non_ref)
    return (ref, non_ref)

In [220]:
print (ref_nonref('TPM3'))

(['TPM3-224'], ['TPM3-201', 'TPM3-202', 'TPM3-203', 'TPM3-204', 'TPM3-205', 'TPM3-206', 'TPM3-207', 'TPM3-208', 'TPM3-209', 'TPM3-210', 'TPM3-211', 'TPM3-212', 'TPM3-213', 'TPM3-214', 'TPM3-215', 'TPM3-216', 'TPM3-217', 'TPM3-218', 'TPM3-219', 'TPM3-220', 'TPM3-221', 'TPM3-222', 'TPM3-223', 'TPM3-225', 'TPM3-226', 'TPM3-227'])


In [170]:
## Read in full blast results
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
blast_df= pd.read_csv('Data/blast_full_PDB_vs_gencode.tsv', sep='\t', header = 1, names = columns)
blast_df

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident
1,7evw_R,599.0,36.0,574.0,FZD1-201|ENST00000287934.4,647.0,103.0,647.0,0.000000e+00,905.0,551.0,440.0,18.0,6.0,93.0,79.855
2,7evw_R,599.0,38.0,574.0,FZD2-201|ENST00000315323.5,565.0,28.0,565.0,0.000000e+00,893.0,553.0,434.0,31.0,3.0,88.0,78.481
3,7evw_R,599.0,49.0,563.0,FZD5-201|ENST00000295417.4,585.0,33.0,536.0,2.420000e-180,523.0,533.0,276.0,47.0,14.0,210.0,51.782
4,7evw_R,599.0,44.0,589.0,FZD10-201|ENST00000229030.5,581.0,29.0,573.0,1.480000e-165,485.0,562.0,261.0,33.0,10.0,268.0,46.441
5,7evw_R,599.0,49.0,562.0,FZD3-201|ENST00000240093.8,666.0,28.0,512.0,2.230000e-162,480.0,515.0,236.0,31.0,3.0,248.0,45.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909511,"2z9a_A,B",88.0,3.0,81.0,PSAP-201|ENST00000394936.8,524.0,311.0,389.0,7.630000e-47,157.0,79.0,79.0,0.0,0.0,0.0,100.000
1909512,"2z9a_A,B",88.0,3.0,55.0,PSAP-204|ENST00000633965.1,167.0,115.0,167.0,4.070000e-31,108.0,53.0,53.0,0.0,0.0,0.0,100.000
1909513,8d58_A,246.0,1.0,246.0,METTL1-202|ENST00000324871.12,276.0,20.0,265.0,0.000000e+00,516.0,246.0,246.0,0.0,0.0,0.0,100.000
1909514,8d58_A,246.0,104.0,240.0,METTL1-204|ENST00000548504.1,124.0,2.0,124.0,3.650000e-85,250.0,137.0,123.0,14.0,1.0,0.0,89.781


### Function to reduce the data to top matches of each PDB
Function is called top_five, the the second option is the numner of hits to include

In [ ]:
def top_five(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            keep.append(True)
        else:
            count += 1
            if count<=max:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])
#top_five_df = top_five(copy_df)
#top_five_df.to_excel('top5_pdb_vs_gencode.xlsx')

In [173]:
def top_five_bitscore(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            bitscore_threshold  = row['bitscore'] * 0.95
            keep.append(True)
        else:
            count += 1
            if count<=max and row['bitscore'] > bitscore_threshold:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])
bitscore_df = top_five_bitscore(blast_df)
bitscore_df
#top_five_df.to_excel('top5_pdb_vs_gencode.xlsx')

## This will find all isoforms that 
def top_five_bitscore_exact(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            bitscore_threshold  = row['bitscore'] 
            keep.append(True)
        else:
            count += 1
            if row['bitscore'] == bitscore_threshold and count<=max:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])

In [174]:
top_100_matched = top_five_bitscore_exact(blast_df, 100)
top_100_matched

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident
1,7evw_R,599.0,36.0,574.0,FZD1-201|ENST00000287934.4,647.0,103.0,647.0,0.000000e+00,905.0,551.0,440.0,18.0,6.0,93.0,79.855
23,2zb4_A,357.0,7.0,357.0,PTGR2-201|ENST00000267568.8,351.0,1.0,351.0,0.000000e+00,728.0,351.0,351.0,0.0,0.0,0.0,100.000
24,2zb4_A,357.0,7.0,357.0,PTGR2-204|ENST00000555228.5,351.0,1.0,351.0,0.000000e+00,728.0,351.0,351.0,0.0,0.0,0.0,100.000
25,2zb4_A,357.0,7.0,357.0,PTGR2-205|ENST00000555661.6,351.0,1.0,351.0,0.000000e+00,728.0,351.0,351.0,0.0,0.0,0.0,100.000
32,7tql_e,51.0,4.0,51.0,FAU-203|ENST00000525297.5,98.0,44.0,98.0,1.560000e-23,85.5,55.0,46.0,7.0,1.0,2.0,83.636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909411,1adq_L,213.0,97.0,213.0,IGLL5-203|ENST00000532223.2,215.0,99.0,215.0,8.160000e-74,223.0,117.0,111.0,0.0,0.0,6.0,94.872
1909412,1adq_L,213.0,97.0,213.0,IGLL5-201|ENST00000526893.6,214.0,98.0,214.0,9.930000e-74,223.0,117.0,111.0,0.0,0.0,6.0,94.872
1909457,4z0x_A,105.0,1.0,92.0,IGLV3-1-201|ENST00000390319.2,115.0,21.0,112.0,6.100000e-60,180.0,92.0,86.0,0.0,0.0,6.0,93.478
1909511,"2z9a_A,B",88.0,3.0,81.0,PSAP-201|ENST00000394936.8,524.0,311.0,389.0,7.630000e-47,157.0,79.0,79.0,0.0,0.0,0.0,100.000


In [178]:
top_100_list = []
for i,row in top_100_matched.iterrows():
    top_100_list.append(row.tolist())
print (top_100_list[0])

<class 'list'>


In [179]:
print (top_100_list[0])

['7evw_R', 599.0, 36.0, 574.0, 'FZD1-201|ENST00000287934.4', 647.0, 103.0, 647.0, 0.0, 905.0, 551.0, 440.0, 18.0, 6.0, 93.0, 79.855]


In [190]:
top_100_matched.to_csv('Data/top_100_matched.tsv', sep='\t')

In [187]:
pdb_to_gencode[0]
#pdb_grouped_by_qseqid

['0',
 '7evw_R',
 '599',
 '38',
 '574',
 'FZD7-201|ENST00000286201.3',
 '574',
 '38',
 '574',
 '0.0',
 '1115',
 '537',
 '537',
 '0',
 '0',
 '0',
 '100.000']

In [213]:
import csv

# Read in the appris transcripts
appris_transcripts = set()
with open('Data/appris_transcript_names.tsv', 'r') as appris_file:
    appris_reader = csv.reader(appris_file, delimiter='\t')
    for row in appris_reader:
        appris_transcripts.add(row[1])
    print (len(appris_transcripts))

# Read in the nonappris transcripts
nonappris_transcripts = set()
with open('Data/not_appris_transcript_names.tsv', 'r') as nonappris_file:
    nonappris_reader = csv.reader(nonappris_file, delimiter='\t')
    for row in nonappris_reader:
        nonappris_transcripts.add(row[1])
    print (len(nonappris_transcripts))

# Read in the PDB to GenCode mapping
pdb_to_gencode = []
with open('Data/top_100_matched_mod.tsv', 'r') as pdb_file:
    pdb_reader = csv.reader(pdb_file, delimiter='\t')
    next(pdb_reader, None)
    for row in pdb_reader:
        row[5] = row[5].split('|')[0]
        pdb_to_gencode.append(row)
    print (len (pdb_to_gencode))
# Group the PDB to GenCode mappings by qseqid
#print (pdb_to_gencode[0])
pdb_grouped_by_qseqid = {}
for row in pdb_to_gencode:
    #print (row)
    qseqid = row[1]
    if qseqid not in pdb_grouped_by_qseqid:
        pdb_grouped_by_qseqid[qseqid] = []
    pdb_grouped_by_qseqid[qseqid].append(row)
print(len(pdb_grouped_by_qseqid))
for k in list(pdb_grouped_by_qseqid.keys())[0:2]:
    print (pdb_grouped_by_qseqid[k])
    


80169
141431
73554
35081
[['1', '7evw_R', '599.0', '36.0', '574.0', 'FZD1-201', '647.0', '103.0', '647.0', '0.0', '905.0', '551.0', '440.0', '18.0', '6.0', '93.0', '79.855']]
[['23', '2zb4_A', '357.0', '7.0', '357.0', 'PTGR2-201', '351.0', '1.0', '351.0', '0.0', '728.0', '351.0', '351.0', '0.0', '0.0', '0.0', '100.0'], ['24', '2zb4_A', '357.0', '7.0', '357.0', 'PTGR2-204', '351.0', '1.0', '351.0', '0.0', '728.0', '351.0', '351.0', '0.0', '0.0', '0.0', '100.0'], ['25', '2zb4_A', '357.0', '7.0', '357.0', 'PTGR2-205', '351.0', '1.0', '351.0', '0.0', '728.0', '351.0', '351.0', '0.0', '0.0', '0.0', '100.0']]


In [214]:
for qseqid, pdb_group in pdb_grouped_by_qseqid.items():
    print (f'qseqid, pdb_group = {qseqid}, {pdb_group}')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



qseqid, pdb_group = 1bzk_A, [['1899888', '1bzk_A', '43.0', '2.0', '43.0', 'SLC4A1-201', '911.0', '389.0', '430.0', '8.69e-22', '86.7', '42.0', '42.0', '0.0', '0.0', '0.0', '100.0']]
qseqid, pdb_group = 1bho_1,2, [['1899889', '1bho_1,2', '190.0', '2.0', '190.0', 'ITGAM-208', '1153.0', '149.0', '337.0', '4.3000000000000005e-129', '390.0', '189.0', '189.0', '0.0', '0.0', '0.0', '100.0'], ['1899890', '1bho_1,2', '190.0', '2.0', '190.0', 'ITGAM-201', '1152.0', '149.0', '337.0', '4.650000000000001e-129', '390.0', '189.0', '189.0', '0.0', '0.0', '0.0', '100.0']]
qseqid, pdb_group = 4cg4_A,B,C,D,E,F, [['1899899', '4cg4_A,B,C,D,E,F', '377.0', '3.0', '370.0', 'MEFV-201', '781.0', '414.0', '781.0', '0.0', '768.0', '368.0', '368.0', '0.0', '0.0', '0.0', '100.0']]
qseqid, pdb_group = 1can_A, [['1900008', '1can_A', '260.0', '2.0', '260.0', 'CA2-201', '260.0', '2.0', '260.0', '0.0', '533.0', '259.0', '259.0', '0.0', '0.0', '0.0', '100.0']]
qseqid, pdb_group = 2myq_A, [['1900069', '2myq_A', '45.0', '2

In [218]:
# Filter the PDB to GenCode mapping based on appris and nonappris transcripts
filtered_pdb_to_gencode = []
for qseqid, pdb_group in pdb_grouped_by_qseqid.items():
    #print (f'qseqid, pdb_group = {qseqid}, {pdb_group}')
    
    # Get the top hit based on bitscore
    top_hit = max(pdb_group, key=lambda row: float(row[10]))
    #print (f'top_hit = {top_hit}')
    #top_bitscore = float(top_hit[10])
    
    # Get all other hits with the same bitscore as the top hit
    other_hits = [row for row in pdb_group if float(row[10]) == top_bitscore and row != top_hit]
    
    # Determine if all sseqids are nonappris-
    all_nonappris = all(row[5] in nonappris_transcripts for row in pdb_group)
    
    # If all sseqids are nonappris, add the gene name to the top hit and output to the filtered list
    if all_nonappris:
        top_hit_with_gene = top_hit + [top_hit[5].split('-')[0]]
        filtered_pdb_to_gencode.append(top_hit_with_gene)
        for other_hit in other_hits:
            other_hit_with_gene = other_hit + [other_hit[5].split('-')[0]]
            filtered_pdb_to_gencode.append(other_hit_with_gene)
print (f'filtered_pdb_to_gencode is {len(filtered_pdb_to_gencode)} long')
# Write out the filtered mapping to a TSV file


filtered_pdb_to_gencode is 8056 long


In [219]:
with open('Data/filtered_pdb_gencode_mapping.tsv', 'w') as output_file:
    output_writer = csv.writer(output_file, delimiter='\t')
    for row in filtered_pdb_to_gencode:
        output_writer.writerow(row)

In [200]:
genes_with_isos = []

[['32',
  '7tql_e',
  '51.0',
  '4.0',
  '51.0',
  'FAU-203',
  '98.0',
  '44.0',
  '98.0',
  '1.56e-23',
  '85.5',
  '55.0',
  '46.0',
  '7.0',
  '1.0',
  '2.0',
  '83.636',
  'FAU'],
 ['488',
  '7e8h_I,K',
  '760.0',
  '5.0',
  '760.0',
  'DPP6-203',
  '801.0',
  '30.0',
  '785.0',
  '0.0',
  '1588.0',
  '756.0',
  '756.0',
  '0.0',
  '0.0',
  '0.0',
  '100.0',
  'DPP6'],
 ['830',
  '2fy2_A',
  '612.0',
  '2.0',
  '612.0',
  'CHAT-204',
  '630.0',
  '2.0',
  '615.0',
  '0.0',
  '1234.0',
  '614.0',
  '596.0',
  '3.0',
  '2.0',
  '15.0',
  '97.068',
  'CHAT'],
 ['831',
  '2fy2_A',
  '612.0',
  '2.0',
  '612.0',
  'CHAT-203',
  '630.0',
  '2.0',
  '615.0',
  '0.0',
  '1234.0',
  '614.0',
  '596.0',
  '3.0',
  '2.0',
  '15.0',
  '97.068',
  'CHAT'],
 ['832',
  '2fy2_A',
  '612.0',
  '2.0',
  '612.0',
  'CHAT-202',
  '630.0',
  '2.0',
  '615.0',
  '0.0',
  '1234.0',
  '614.0',
  '596.0',
  '3.0',
  '2.0',
  '15.0',
  '97.068',
  'CHAT'],
 ['922',
  '6b0b_A,E',
  '186.0',
  '4.0',
  '186.

## Identify genes with more than one isoform. 

In [198]:
genes_dict={}
def multiple_isos(df):
    genes=defaultdict(set)
    # Build a dictionary where the values are a set of isoforms for each gene
    for id in df['sseqid']:
        try:
            id = id.split('|')[0]
            parts = id.split('-')
            gene='-'.join(parts[0:-1])
            iso=parts[-1]
            #print (id, gene, iso)
            genes[gene].add(iso)
        except:
            pass
    # Create a list (check) of genes that have more than 1 isoform represented.
    check = []
    for g,g_set in genes.items():
        if len(g_set) > 1:
            check.append(g)
            genes_dict[g]=g_set
    print (f'check is {len(check)} long')

    # Create list of df indicies of all genes that have more than one df 
    found=[]
    for i,row in df.iterrows():
        #print (row['sseqid'].split('-')[0])
        try:
            if row['sseqid'].split('-')[0] in check:
                found.append(i)
        except:
            pass
    #print (f'found is {found}')
    found_df = df.loc[found]
    found_df.sort_values(by=['sseqid'], inplace=True)
    return found_df

found_df = multiple_isos(top_100_matched)
found_df

check is 3751 long


,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident
736022,2cpd_A,99.0,8.0,93.0,A1CF-202|ENST00000373995.7,594.0,231.0,316.0,3.460000e-52,173.0,86.0,86.0,0.0,0.0,0.0,100.000
736023,2cpd_A,99.0,8.0,93.0,A1CF-205|ENST00000395489.7,602.0,231.0,316.0,3.850000e-52,173.0,86.0,86.0,0.0,0.0,0.0,100.000
736021,2cpd_A,99.0,8.0,93.0,A1CF-206|ENST00000395495.6,594.0,231.0,316.0,3.460000e-52,173.0,86.0,86.0,0.0,0.0,0.0,100.000
1326122,"2vgz_A,B",427.0,4.0,427.0,AADAT-201|ENST00000337664.9,425.0,2.0,425.0,0.000000e+00,878.0,424.0,424.0,0.0,0.0,0.0,100.000
1347443,"2qlr_A,B,C,D",425.0,1.0,425.0,AADAT-201|ENST00000337664.9,425.0,1.0,425.0,0.000000e+00,880.0,425.0,425.0,0.0,0.0,0.0,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771849,"5mkw_A,B",122.0,3.0,122.0,ZRANB3-202|ENST00000401392.5,1077.0,946.0,1065.0,1.450000e-80,256.0,120.0,120.0,0.0,0.0,0.0,100.000
1771848,"5mkw_A,B",122.0,3.0,122.0,ZRANB3-210|ENST00000536680.5,625.0,494.0,613.0,2.310000e-83,256.0,120.0,120.0,0.0,0.0,0.0,100.000
1771847,"5mkw_A,B",122.0,3.0,122.0,ZRANB3-211|ENST00000619650.4,598.0,411.0,530.0,9.090000e-84,256.0,120.0,120.0,0.0,0.0,0.0,100.000
748025,2cot_A,77.0,5.0,74.0,ZSCAN16-201|ENST00000340487.5,348.0,222.0,291.0,1.840000e-42,141.0,70.0,66.0,0.0,0.0,4.0,94.286


### Find genes that have a structures that match ref and non-ref isoforms 
Note: genes_dict will only contain genes that match more than one isoform.

In [119]:
to_inspect=[]
#for k in list(genes_dict.keys())[0:10]:
for k in genes_dict:
    try:
        r,nr=ref_nonref(k)
        r_iso = r.split('-')[1]
        if r_iso in genes_dict[k]:
            to_inspect.append(k)
    except Exception as e:
        pass

    

### Function to extract sequences of Isoforms and related sequences
USE: get_gene_seqs (gene, df=found_df)  
By default, will extract data from found_df, but can be specified 

In [143]:
pdb_file='Data/pdb_unique_human_seqs_230301.fa'
gene_file='Data/gencode_v43_translations_mod.fa'  
def extract_sequence(filename, sequence_id):
    sequence = ""
    with open(filename, "r") as f:
        for line in f:
            if line.startswith(">") and sequence_id in line:
                sequence += line
                # Found the sequence ID we want
                while True:
                    line = f.readline().strip()
                    if line.startswith(">") or not line:
                        # Reached the end of the sequence
                        break
                    sequence += line
    sequence += "\n"
    return sequence
def get_gene_seqs (gene, df=found_df):
    matches = found_df.loc[ found_df['sseqid'].str.contains(gene) ]
    isos = set()
    pdbs = set()
    for i,row in matches.iterrows():
        isos.add(row['sseqid'])
        pdbs.add(row['qseqid'])
    seqs=""
    for pdb in pdbs:
        seqs+= extract_sequence (pdb_file, pdb)
    for iso in isos:
        seqs+= extract_sequence (gene_file, iso)
    return (seqs)

s=get_gene_seqs('EPHA7')
print (s)

>7eec_A
HFKFPGTKTYIDPETYEDPNRAVHQFAKELDASCIKIERVIGAGEFGEVCSGRLKLPRKRDVAVAIKTLKVGYTEKQRRDFLCEASIMGQFDHPNVVHLEGVVTRGKPVMIVIEFMENGALDAFLRKHDGQFTVIQLVGMLRGIAAGMRYLADMGYVHRDLAARNILVNSNLVCKVSDFGLSRVIEDDPEAVYTTTGGKIPVRWTAPEAIQYRKFTSASDVWSYGIVMWEVMSYGERPYWDMSNQDVIKAIEEGYRLPAPMDCPAGLHQLMLDCWQKERAERPKFEQIVGILDKMIRNPNSAHHH
>3nru_A,B,C,D,E,F,G,H,I,J,K,L
AAEVLLLDSKAQQTELEWISSPPNGWEEISGLDENYTPIRTYQVCQVMEPNQNNWLRTNWISKGNAQRIFVELKFTLRDCNSLPGVLGTCKETFNLYYYETDYDTGRNIRENLYVKIDTIAADESFTQGDLGERKMKLNTEVREIGPLSKKGFYLAFQDVGACIALVSVKVYYKKEFVEHHHHHHHH
>7eed_A
HFKFPGTKTYIDPETYEDPNRAVHQFAKELDASCIKIERVIGAGEFGEVCSGRLKLPGKRDVAVAIKTLKVGYTEKQRRDFLCEASIMGQFDHPNVVHLEGVVTRGKPVMIVIEFMENGALDAFLRKHDGQFTVIQLVGMLRGIAAGMRYLAHMGYVHRDLAARNILVNSNLVCKVSDFGLSRVIEDDPEAVYTTTGGKIPVRWTAPEAIQYRKFTSASDVWSYGIVMWEVMSYGERPYWDMSNQDVIKAIEEGYRLPAPMDCPAGLHQLMLDCWQKERAERPKFEQIVGILDKMIRNPNSAHH
>3h8m_A,B
MHHHHHHSSGRENLYFQGTPDFTTFCSVGEWLQAIKMERYKDNFTAAGYNSLESVARMTIEDVMSLGITLVGHQKKIMSSIQTMRAQMLH
>7eef_A
HFKFPGTKTYIDPETYEDPNRAVHQFAKELDASCIKIERVIGAGEF

In [141]:
found_df

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident
736022,2cpd_A,99.0,8.0,93.0,A1CF-202|ENST00000373995.7,594.0,231.0,316.0,3.460000e-52,173.0,86.0,86.0,0.0,0.0,0.0,100.000
736025,2cpd_A,99.0,8.0,93.0,A1CF-204|ENST00000374001.6,586.0,223.0,308.0,5.170000e-52,172.0,86.0,86.0,0.0,0.0,0.0,100.000
736023,2cpd_A,99.0,8.0,93.0,A1CF-205|ENST00000395489.7,602.0,231.0,316.0,3.850000e-52,173.0,86.0,86.0,0.0,0.0,0.0,100.000
736021,2cpd_A,99.0,8.0,93.0,A1CF-206|ENST00000395495.6,594.0,231.0,316.0,3.460000e-52,173.0,86.0,86.0,0.0,0.0,0.0,100.000
736024,2cpd_A,99.0,8.0,93.0,A1CF-207|ENST00000414883.2,586.0,223.0,308.0,5.170000e-52,172.0,86.0,86.0,0.0,0.0,0.0,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745150,2yum_A,75.0,8.0,71.0,ZZZ3-201|ENST00000370798.5,409.0,158.0,221.0,2.250000e-37,129.0,64.0,62.0,0.0,0.0,2.0,96.875
747357,2fc7_A,82.0,8.0,76.0,ZZZ3-201|ENST00000370798.5,409.0,313.0,381.0,4.880000e-47,155.0,69.0,69.0,0.0,0.0,0.0,100.000
548535,6e83_B,64.0,4.0,64.0,ZZZ3-202|ENST00000370801.8,903.0,814.0,874.0,8.530000e-39,136.0,61.0,60.0,0.0,0.0,1.0,98.361
745151,2yum_A,75.0,8.0,71.0,ZZZ3-202|ENST00000370801.8,903.0,652.0,715.0,4.300000e-37,132.0,64.0,62.0,0.0,0.0,2.0,96.875


In [ ]:
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
filtered = pd.read_csv('Data/filtered_pdb_gencode_mapping.tsv', sep='\t', index_col= False, header = None, names = columns)
filtered

### Create a directory with sequence files for all the matches

In [33]:
gene_dict= defaultdict(set)
def seq_loop(df):
    gene_list = []
    for id in df['sseqid']:
        try:
            id = id.split('|')[0]
            parts = id.split('-')
            gene='-'.join(parts[0:-1])
            if gene not in gene_list:
                gene_list.append(gene)
                gene_dict[gene].add(id)
                file = f'Data/Seq_groups/{gene}.fa'
                if os.path.isfile(file):
                    continue
                s = get_gene_seqs(gene)
                with open (f'Data/Seq_groups/{gene}.fa', 'w') as out:
                    out.write(s)
        except:
            pass
    return (gene_list)
gene_list=seq_loop(found_df)
#filtered_genes= pd.DataFrame(done)  
    

In [153]:
def show_msa(gene_list, fresh=0, last=None):
    '''Will generate and show a MSA for a list of genes. If last is set, it will skip until it reaches that gene'''
    if len(gene_list)==1:
        fresh =1
    for g in gene_list:
        if fresh==0 and g!=last:
            continue
        print (g)
        print (f'Reference isoform is {ref_nonref(g)[0]}')
        fresh=1
        last = g
        fasta_file = f'Data/Seq_groups/{g}.fa'
        if not os.path.isfile(fasta_file):
            get_gene_seqs(g)
        msa_file = f'Data/Seq_groups/{g}.msa'
        if not os.path.isfile(msa_file):
            cmd = f'clustalo -i Data/Seq_groups/{g}.fa  --outfmt=clu -o {msa_file}'
            print (cmd)
            r=subprocess.getoutput(cmd)
            if r[0:5] =="ERROR":
                print (f'Something went wrong with {g}')
                continue
            print (r)
        with open (msa_file, 'r') as f:
            rr=f.read()
            print (rr)

        #read = input ("Ready to move on? f to see sequence file ")
        #if read.lower() == 'f':
            with open (f'Data/Seq_groups/{g}.fa') as seqs:
                print (seqs.read())
            if len (gene_list) > 1:
                read = input ("Ready to move on? ")
            
def show_seqs_loop(gene_list, fresh=0, last=None):
    #expects a df with the name of the gene in position 0
    for g in gene_list:
        #print (g)
        if fresh==0 and g!=last:
            continue
        fresh=1
        last = g   
        with open (f'Data/Seq_groups/{g}.fa') as seqs:
            print(seqs.read())
        read = input ("Ready to move on? f to see sequence file ")

def show_seqs(gene):
    with open (f'Data/Seq_groups/{gene}.fa') as seqs:
        print (seqs.read())

def matches_for_gene(gene, df = found_df):
    df2 = df.loc[df['sseqid'].str.startswith(gene)]
    return (df2)

In [144]:
show_msa(to_inspect, fresh =0, last='EPHA7')

EPHA7
Reference isoform is EPHA7-202
CLUSTAL O(1.2.4) multiple sequence alignment


7eed_A                            ------------------------------------------------------------
3nru_A,B,C,D,E,F,G,H,I,J,K,L      -----------------------------AAEVLLLDSKAQQTELEWISSPPNGWEEISG
7eec_A                            ------------------------------------------------------------
2rei_A                            ------------------------------------------------------------
7eef_A                            ------------------------------------------------------------
3h8m_A,B                          ------------------------------------------------------------
EPHA7-201|ENST00000369297.1       MVFQTRYPSWIILCYIWLLRFAHTGEAQAAKEVLLLDSKAQQTELEWISSPPNGWEEISG
EPHA7-206|ENST00000680190.1       ------------------------------------------------------------
                                                                                              

7eed_A                            --------------------------

KeyboardInterrupt: Interrupted by user

In [157]:
show_msa(['HARS1'])

HARS1
Reference isoform is HARS1-207
CLUSTAL O(1.2.4) multiple sequence alignment


5w6m_A,B                         ------------------------------------------------------------
4phc_A,B,C,D                     -----------MAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLGPDE
2lw7_A                           ------------------------------------------------------------
4g85_A,B                         GPGSEFELRRQMAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLGPDE
4g84_A,B                         GPGS-----------------------------------------------------EFE
1x59_A                           ----GSSGSSGMAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLGPDE
HARS1-214|ENST00000643686.1      -----------MAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLGPDE
HARS1-211|ENST00000509087.2      ------------------------------------------------------------
HARS1-207|ENST00000504156.7      -----------MAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLGPDE
                                                                      

In [165]:
functional=pd.read_csv('Data/APPRIS_FunctionalTableLong.tsv',sep ='\t')
functional_genes = functional['HGNC'].tolist()
functional_genes
show_msa(functional_genes,fresh=0, last='ITGA7')


['ABI2',
 'ACOX1',
 'ACTN1',
 'ACTN1',
 'AP1B1',
 'AP1S2',
 'ATP2B1',
 'ATP2B4',
 'C2CD5',
 'CALD1',
 'CAMK2G',
 'CAMK2G',
 'CAMSAP3',
 'CDC42',
 'DNM1L',
 'EPB41L2',
 'FYN',
 'GLS',
 'H2AFY',
 'ITGA7',
 'MPRIP',
 'MYH10',
 'MYL6',
 'NECTIN2',
 'OGDH',
 'PDLIM3',
 'PDLIM5',
 'PKM',
 'PPP1R12B',
 'RABGAP1L',
 'RTN4',
 'SH3GLB2',
 'SLC25A3',
 'SLC44A2',
 'SORBS2',
 'SPTAN1',
 'TPM1',
 'TPM1',
 'TPM1',
 'TPM2',
 'TPM2',
 'TPM3',
 'TPM3',
 'TPM3',
 'TPM4']

In [166]:
functional=pd.read_csv('Data/Tandem_exon_derived_substitution_HES_APPRIS_functional.tsv',sep ='\t')
functional_genes = functional['HGNC'].tolist()
functional_genes
show_msa(functional_genes,fresh=1)


ABCC9
Reference isoform is ABCC9-201
clustalo -i Data/Seq_groups/ABCC9.fa  --outfmt=clu -o Data/Seq_groups/ABCC9.msa
Something went wrong with ABCC9
ABCD3
Reference isoform is ABCD3-202
clustalo -i Data/Seq_groups/ABCD3.fa  --outfmt=clu -o Data/Seq_groups/ABCD3.msa
Something went wrong with ABCD3
ACAN
Reference isoform is ACAN-207
clustalo -i Data/Seq_groups/ACAN.fa  --outfmt=clu -o Data/Seq_groups/ACAN.msa
Something went wrong with ACAN
ACOX1
Reference isoform is ACOX1-202
CLUSTAL O(1.2.4) multiple sequence alignment


7q84_A,B                          MHHHHHHMNPDLRRERDSASFNPELLTHILDGSPEKTRRRREIENMILNDPDFQHEDLNF
ACOX1-201|ENST00000293217.10      -------MNPDLRRERDSASFNPELLTHILDGSPEKTRRRREIENMILNDPDFQHEDLNF
ACOX1-202|ENST00000301608.9       -------MNPDLRRERDSASFNPELLTHILDGSPEKTRRRREIENMILNDPDFQHEDLNF
                                         *****************************************************

7q84_A,B                          LTRSQRYEVAVRKSAIMVKKMREFGIADPDEIMWFKNFVHRGRPEPLDLHLGMFLPTLL

Ready to move on? 
ACSL1
Reference isoform is ACSL1-201
clustalo -i Data/Seq_groups/ACSL1.fa  --outfmt=clu -o Data/Seq_groups/ACSL1.msa
Something went wrong with ACSL1
ACSL6
Reference isoform is ACSL6-231
clustalo -i Data/Seq_groups/ACSL6.fa  --outfmt=clu -o Data/Seq_groups/ACSL6.msa
Something went wrong with ACSL6
ACTN1
Reference isoform is ACTN1-203
CLUSTAL O(1.2.4) multiple sequence alignment


2eyi_A                           -------------------------AGHMEKQQRKTFTAWCNSHLRKAGTQIENIEEDFR
2n8y_A                           ------------------------------------------------------------
ACTN1-236|ENST00000683267.1      MDHYDSQQTNDYMQPEEDWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIENIEEDFR
ACTN1-241|ENST00000684182.1      ------------------------------------------------------------
                                                                                             

2eyi_A                           DGLKLMLLLEVISGERLAKPERGKMRVHKISNVNKALDFIASKGVKLVSIGAEEIVDGNV
2n8y_A                           -

Ready to move on? 
ACTN2
Reference isoform is ACTN2-201
CLUSTAL O(1.2.4) multiple sequence alignment


5a4b_A,B                         ------------------YMIQEEEWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIE
1h8b_A                           ------------------------------------------------------------
3lue_K,M,L,O,N,Q,P,S,R,T         ----------------------------------AWEKQQRKTFTAWCNSHLRKAGTQIE
1quu_A                           ------------------------------------------------------------
7ank_B                           ------------------------------------------------------------
6ts3_A,B                         ------------------------------------------------------------
6swt_A                           ----------------GPYMIQEEEWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIE
1hci_A,B                         ------------------------------------------------------------
5a36_A,B                         ----------------GPYMIQEEEWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIE
5a37_A,B                         ----------------GP

Ready to move on? 
ADAM23
Reference isoform is ADAM23-201
clustalo -i Data/Seq_groups/ADAM23.fa  --outfmt=clu -o Data/Seq_groups/ADAM23.msa
Something went wrong with ADAM23
AIFM1
Reference isoform is AIFM1-229
CLUSTAL O(1.2.4) multiple sequence alignment


4fdc_B                           ------------------------------------------------------------
5fs7_A,B                         ------------------------------------------------------------
5fs8_A                           ------------------------------------------------------------
5kvh_A,B                         ------------------------------------------------------------
5kvi_A                           ------------------------------------------------------------
4bur_A,B,C,D                     ------------------------------------------------------------
5fs9_A,B                         ------------------------------------------------------------
5fmh_A                           ----------------------------------------------------

Ready to move on? 
AKAP7
Reference isoform is AKAP7-204
CLUSTAL O(1.2.4) multiple sequence alignment


3j4q_A                           MERPAAGEIDANKCDHLSRGEEGTGDLETSPVGSLADLPFAAVDIQDDCGLPDVPQGNVP
5jj2_A                           ------------------------------------------------------------
4zp3_M,N,O,P,Q,R                 ------------------------------------------------------------
2vfk_A                           ------------------------------------------------------------
AKAP7-204|ENST00000431975.7      MERPEAGGINSNECENVSRKKKMSEEFEANTMDSLVDMPFATVDIQDDCGITDEPQINL-
AKAP7-201|ENST00000263050.3      ------------------------------------------------------------
                                                                                             

3j4q_A                           QGNPKRSKENRGDRNDHVKKRKKAKKDYQPNYFLSIPITNKKITAGIKVLQNSILRQDNR
5jj2_A                           --------------------RKKKRKDYQPNYFLSIPITNKEIIKGIKILQNAIIQQDER
4zp3_M,N,O,P,Q,R                 -----------------

...

Ready to move on? 
IDH3B
Reference isoform is IDH3B-202
clustalo -i Data/Seq_groups/IDH3B.fa  --outfmt=clu -o Data/Seq_groups/IDH3B.msa

CLUSTAL O(1.2.4) multiple sequence alignment


8grb_C,D,F,H,J,L,N,P             ----------------------------------ASRSQAEDVRVEGSFPVTMLPGDGVG
6kde_B,D                         ----------------------------------ASRSQAEDVRVEGSFPVTMLPGDGVG
7ce3_D                           -------------------------------MGSASRSQAEDVRVEGSFPVTMLPGDGVG
IDH3B-202|ENST00000380851.9      MAALSGVRWLTRALVSAGNPGAWRGLSTSAAAHAASRSQAEDVRVEGSFPVTMLPGDGVG
IDH3B-201|ENST00000380843.9      MAALSGVRWLTRALVSAGNPGAWRGLSTSAAAHAASRSQAEDVRVEGSFPVTMLPGDGVG
                                                                   **************************

8grb_C,D,F,H,J,L,N,P             PELMHAVKEVFKAAAVPVEFQEHHLSEVQNMASEEKLEQVLSSMKENKVAIIGKIHTPME
6kde_B,D                         PELMHAVKEVFKAAAVPVEFQEHHLSEVQNMASEEKLEQVLSSMKENKVAIIGKIHTPME
7ce3_D                           PELMHAVKEVFKAAAVPVEFQEHHLSEVQN

Ready to move on? 
ITGA3
Reference isoform is ITGA3-202
clustalo -i Data/Seq_groups/ITGA3.fa  --outfmt=clu -o Data/Seq_groups/ITGA3.msa
Something went wrong with ITGA3
ITGA6
Reference isoform is ITGA6-202
clustalo -i Data/Seq_groups/ITGA6.fa  --outfmt=clu -o Data/Seq_groups/ITGA6.msa

CLUSTAL O(1.2.4) multiple sequence alignment


7ceb_A                            -----------------------FNLDTREDNVIRKYGDPGSLFGFSLAMHWQLQPEDKR
ITGA6-201|ENST00000264107.12      MAAAGQLCLLYLSAGLLSRLGAAFNLDTREDNVIRKYGDPGSLFGFSLAMHWQLQPEDKR
ITGA6-212|ENST00000684293.1       MAAAGQLCLLYLSAGLLSRLGAAFNLDTREDNVIRKYGDPGSLFGFSLAMHWQLQPEDKR
                                                         *************************************

7ceb_A                            LLLVGAPRAEALPLQRANRTGGLYSCDITARGPCTRIEFDNDADPTSESKEDQWMGVTVQ
ITGA6-201|ENST00000264107.12      LLLVGAPRAEALPLQRANRTGGLYSCDITARGPCTRIEFDNDADPTSESKEDQWMGVTVQ
ITGA6-212|ENST00000684293.1       LLLVGAPRAEALPLQRANRTGGLYSCDITARGPCTRIEFDNDADPTSESKEDQWMGVTVQ
 

Ready to move on? 
ITGA6
Reference isoform is ITGA6-202
CLUSTAL O(1.2.4) multiple sequence alignment


7ceb_A                            -----------------------FNLDTREDNVIRKYGDPGSLFGFSLAMHWQLQPEDKR
ITGA6-201|ENST00000264107.12      MAAAGQLCLLYLSAGLLSRLGAAFNLDTREDNVIRKYGDPGSLFGFSLAMHWQLQPEDKR
ITGA6-212|ENST00000684293.1       MAAAGQLCLLYLSAGLLSRLGAAFNLDTREDNVIRKYGDPGSLFGFSLAMHWQLQPEDKR
                                                         *************************************

7ceb_A                            LLLVGAPRAEALPLQRANRTGGLYSCDITARGPCTRIEFDNDADPTSESKEDQWMGVTVQ
ITGA6-201|ENST00000264107.12      LLLVGAPRAEALPLQRANRTGGLYSCDITARGPCTRIEFDNDADPTSESKEDQWMGVTVQ
ITGA6-212|ENST00000684293.1       LLLVGAPRAEALPLQRANRTGGLYSCDITARGPCTRIEFDNDADPTSESKEDQWMGVTVQ
                                  ************************************************************

7ceb_A                            SQGPGGKVVTCAHRYEKRQHVNTKQESRDIFGRCYVLSQNLRIEDDMDGGDWSFCDGRLR
ITGA6-201|ENST00000264107.12      SQGPGG

Ready to move on? 
ITGA7
Reference isoform is ITGA7-213
clustalo -i Data/Seq_groups/ITGA7.fa  --outfmt=clu -o Data/Seq_groups/ITGA7.msa
Something went wrong with ITGA7
ITGB1
Reference isoform is ITGB1-201
clustalo -i Data/Seq_groups/ITGB1.fa  --outfmt=clu -o Data/Seq_groups/ITGB1.msa

CLUSTAL O(1.2.4) multiple sequence alignment


7nwl_B                                                                    ----------------------------QTDENRCLKANAKSCGECIQAGPNCGWCTNST
4dx8_A,B,D,E                                                              ------------------------------------------------------------
4wjk_B                                                                    ----------------------------QTDENRCLKANAKSCGECIQAGPNCGWCTNST
3g9w_C,D                                                                  ------------------------------------------------------------
3vi3_B,D                                                                  ----------------------------QTDENRCLKANAKSCGECIQAGPNC

KeyboardInterrupt: Interrupted by user

#  Need to look at PDBs that do not match Appris, but do match non-appris

example is 

mirage, exonerate


In [221]:
blast_df

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident
1,7evw_R,599.0,36.0,574.0,FZD1-201|ENST00000287934.4,647.0,103.0,647.0,0.000000e+00,905.0,551.0,440.0,18.0,6.0,93.0,79.855
2,7evw_R,599.0,38.0,574.0,FZD2-201|ENST00000315323.5,565.0,28.0,565.0,0.000000e+00,893.0,553.0,434.0,31.0,3.0,88.0,78.481
3,7evw_R,599.0,49.0,563.0,FZD5-201|ENST00000295417.4,585.0,33.0,536.0,2.420000e-180,523.0,533.0,276.0,47.0,14.0,210.0,51.782
4,7evw_R,599.0,44.0,589.0,FZD10-201|ENST00000229030.5,581.0,29.0,573.0,1.480000e-165,485.0,562.0,261.0,33.0,10.0,268.0,46.441
5,7evw_R,599.0,49.0,562.0,FZD3-201|ENST00000240093.8,666.0,28.0,512.0,2.230000e-162,480.0,515.0,236.0,31.0,3.0,248.0,45.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909511,"2z9a_A,B",88.0,3.0,81.0,PSAP-201|ENST00000394936.8,524.0,311.0,389.0,7.630000e-47,157.0,79.0,79.0,0.0,0.0,0.0,100.000
1909512,"2z9a_A,B",88.0,3.0,55.0,PSAP-204|ENST00000633965.1,167.0,115.0,167.0,4.070000e-31,108.0,53.0,53.0,0.0,0.0,0.0,100.000
1909513,8d58_A,246.0,1.0,246.0,METTL1-202|ENST00000324871.12,276.0,20.0,265.0,0.000000e+00,516.0,246.0,246.0,0.0,0.0,0.0,100.000
1909514,8d58_A,246.0,104.0,240.0,METTL1-204|ENST00000548504.1,124.0,2.0,124.0,3.650000e-85,250.0,137.0,123.0,14.0,1.0,0.0,89.781


In [223]:
blast_df.groupby(['qseqid'])['sseqid'].aggregate(lambda x: list(x)).reset_index()

,qseqid,sseqid
0,"10gs_A,B","[GSTP1-202|ENST00000398606.10, GSTP1-201|ENST0..."
1,"11gs_A,B","[GSTP1-202|ENST00000398606.10, GSTP1-201|ENST0..."
2,121p_A,"[HRAS-205|ENST00000451590.5, HRAS-201|ENST0000..."
3,12ca_A,"[CA2-201|ENST00000285379.10, CA13-201|ENST0000..."
4,133l_A,"[LYZ-201|ENST00000261267.7, LYZ-203|ENST000005..."
...,...,...
35149,Warning: [blastp] Query_35901 5nw4_A: Could no...,[nan]
35150,Warning: [blastp] Query_35902 5nvs_A: Could no...,[nan]
35151,"Warning: [blastp] Query_35903 5flc_A,E: Could ...",[nan]
35152,"Warning: [blastp] Query_35904 5nvu_A,B: Could ...",[nan]
